In [8]:
import numpy as np
import pandas as pd
import sklearn as sl
import random

In [9]:
columns = ["match_id", "player_slot", "chat_key", "unit", "radiant_win"]
chat_data = pd.read_csv('match_chat.csv', names=columns, encoding = "ISO-8859-1")
chat_data.head()

,match_id,player_slot,chat_key,unit,radiant_win
0,5008188550,128,yes you are girl,5,0
1,5008188550,128,xoxo,5,0
2,5008188550,1,stop,1,0
3,5008188550,1,I cant love the enemy,1,0
4,5008188550,130,I'd paint u,7,0


In [10]:
len(chat_data)

4709